In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import math
from scipy.stats import norm
from scipy.stats import sem

In [3]:
from statsmodels.stats.anova import AnovaRM
import scikit_posthocs as sp

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="white")
from openpyxl import load_workbook

In [5]:
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.api as sm
import researchpy as rp

In [6]:
import os

1st cohort mice (male n=6, excluded mice X-L-7 cannot pass stage 3 and mice W-N-4 finished stage 3 when the other mice finished the task, since they had difficulty to learn the CPT task; female n=4) and 2nd cohort mice (male n=15, female n=7 excluded W-N-4, A-RL-3 jumper, excluded in this cohort)

#### combine 1st and 2nd cohort TOT data

In [11]:
#### load 1st cohort data
df_list=["...\\d2.csv", "...\\d3.csv", "...\\d5.csv"] #replace the "..." to a full directory of abetii raw data

df_1=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
    df_1=pd.concat([df_1, df_], ignore_index=True)
df_1['cohort']="coh1"

In [12]:
#### load 2nd cohort data
df_list=["...\\d1.csv", "...\\d4.csv"]#replace the "..." to a full directory of abetii raw data

df_2=pd.DataFrame()
for i in df_list:
    df_=pd.read_csv(i)
    if len(df_)<8:
        print(i)
    df_2=pd.concat([df_2, df_], ignore_index=True)
df_2['cohort']="coh2"

In [13]:
### combine 1st and 2nd cohort data
df=pd.concat([df_1, df_2], ignore_index=True)

In [14]:
# convert short time to datetime format
df.loc[:, "Schedule run date short"]=pd.to_datetime(df["Schedule run date"]).dt.date
df["Schedule run date short"]=pd.to_datetime(df["Schedule run date short"], format="%Y-%m-%d")


# add key by using id and run time
df.loc[:, "key"]=df['Group ID'] + "-" + df["Animal ID"] + "-" + df["Schedule run date short"].astype('str') 
df.loc[:, "id"]=df['Group ID'] + "-" + df["Animal ID"]+ "-" + df['cohort']

In [15]:
df.loc[:, "gander"]=np.where(df['Group ID'].isin(["W", "X", "Y", "Z"]), "male", "female")

In [16]:
#df_tot=df[df['dosage']=='tot']
df_tot=df

In [17]:
# test whether include duplicated row
t=df[df.duplicated('key', keep=False)].sort_values(by='key')
t["key"].unique()

array([], dtype=object)

### calculated d' and implus parameter

In [18]:
def hr_calcu(hit, miss):
    hr_rate = hit/(hit+miss)
    return hr_rate

def fr_calcu(mistake, correct_rejection):
    fr_rate = mistake/(mistake+correct_rejection)
    return fr_rate

def d_calcu(hr,fr):
    d=norm.ppf(hr)-norm.ppf(fr)
    return d  

def si_calcu(hr,fr):
    si=(hr-fr)/(2*(hr+fr)-pow((hr+fr), 2))
    return si

def c_calcu(hr,fr):
    c=-(norm.ppf(hr)+norm.ppf(fr))/2
    return c

def ri_calcu(hr,fr):
    ri=(hr+fr-1)/(1-pow((hr-fr), 2))
    return ri

def rp_calcu(centre_ITI_Touches, hit, miss, mistake, correct_rejection, Correction_Trial_Correct_Rejection, Correction_Trial_Mistakes):
    rp=100*(centre_ITI_Touches)/(hit+miss+mistake+correct_rejection+Correction_Trial_Correct_Rejection+Correction_Trial_Mistakes)
    return rp

In [19]:
# #another way to replace "space" in the column name
column_name_list = df_tot.columns.tolist()
new_column=[]
for i in column_name_list:
    i_=i.replace(' ','_')
    new_column.append(i_)

df_tot.columns=new_column
# first_20_session['Environment_name']

In [20]:
df_tot.rename(columns={'Correct_Choice_Latency_Bin_1_(1-15min)': 'corr_latency_bin_1',
 'Correct_Choice_Latency_Bin_2_(15-30min)': 'corr_latency_bin_2',
 'Correct_Choice_Latency_Bin_3_(30-45min)': 'corr_latency_bin_3',
 'Correct_Choice_Latency_Bin_4_(45-60min)': 'corr_latency_bin_4',
 'Correct_Choice_Latency_Bin_5_(60-75min)': 'corr_latency_bin_5',
 'Correct_Choice_Latency_Bin_6_(75-90min)': 'corr_latency_bin_6',
 'Incorrect_Choice_Latency_Bin_1_(1-15min)': 'incorr_latency_bin_1',
 'Incorrect_Choice_Latency_Bin_2_(15-30min)': 'incorr_latency_bin_2',
 'Incorrect_Choice_Latency_Bin_3_(30-45min)': 'incorr_latency_bin_3',
 'Incorrect_Choice_Latency_Bin_4_(45-60min)': 'incorr_latency_bin_4',
 'Incorrect_Choice_Latency_Bin_5_(60-75min)': 'incorr_latency_bin_5',
 'Incorrect_Choice_Latency_Bin_6_(75-90min)': 'incorr_latency_bin_6',
 'Reward_Retrieval_Latency_Bin_1_(0-15min)':'reward_latency_bin_1',
 'Reward_Retrieval_Latency_Bin_2_(15-30min)':'reward_latency_bin_2',
 'Reward_Retrieval_Latency_Bin_3_(30-45min)':'reward_latency_bin_3',
 'Reward_Retrieval_Latency_Bin_4_(45-60min)':'reward_latency_bin_4',
 'Reward_Retrieval_Latency_Bin_5_(60-75min)':'reward_latency_bin_5',
 'Reward_Retrieval_Latency_Bin_6_(75-90min)':'reward_latency_bin_6',
 'Correct_Choice_Latency_Bin_1_(1-15min)_sd': 'corr_latency_bin_1_sd',
 'Correct_Choice_Latency_Bin_2_(15-30min)_sd': 'corr_latency_bin_2_sd',
 'Correct_Choice_Latency_Bin_3_(30-45min)_sd': 'corr_latency_bin_3_sd',
 'Correct_Choice_Latency_Bin_4_(45-60min)_sd': 'corr_latency_bin_4_sd',
 'Correct_Choice_Latency_Bin_5_(60-75min)_sd': 'corr_latency_bin_5_sd',
 'Correct_Choice_Latency_Bin_6_(75-90min)_sd': 'corr_latency_bin_6_sd',
 'Incorrect_Choice_Latency_Bin_1_(1-15min)_sd': 'incorr_latency_bin_1_sd',
 'Incorrect_Choice_Latency_Bin_2_(15-30min)_sd': 'incorr_latency_bin_2_sd',
 'Incorrect_Choice_Latency_Bin_3_(30-45min)_sd': 'incorr_latency_bin_3_sd',
 'Incorrect_Choice_Latency_Bin_4_(45-60min)_sd': 'incorr_latency_bin_4_sd',
 'Incorrect_Choice_Latency_Bin_5_(60-75min)_sd': 'incorr_latency_bin_5_sd',
 'Incorrect_Choice_Latency_Bin_6_(75-90min)_sd': 'incorr_latency_bin_6_sd',
 'Reward_Retrieval_Latency_Bin_1_(0-15min)_sd': 'reward_latency_bin_1_sd',
 'Reward_Retrieval_Latency_Bin_2_(15-30min)_sd': 'reward_latency_bin_2_sd',
 'Reward_Retrieval_Latency_Bin_3_(30-45min)_sd': 'reward_latency_bin_3_sd',
 'Reward_Retrieval_Latency_Bin_4_(45-60min)_sd': 'reward_latency_bin_4_sd',
 'Reward_Retrieval_Latency_Bin_5_(60-75min)_sd': 'reward_latency_bin_5_sd',
 'Reward_Retrieval_Latency_Bin_6_(75-90min)_sd': 'reward_latency_bin_6_sd'}, inplace=True)

In [21]:
drop_col=['15_min_bin_-_No_of_non_correction_trials_(7)',
 '15_min_bin_-_Left_ITI_Touches_(7)',
 '15_min_bin_-_Centre_ITI_Touches_(7)',
 '15_min_bin_-_Right_ITI_Touches_(7)',
 '15_min_bin_-_Blank_Touches_while_image_shown_(7)',
 '15_min_bin_-_Hits_(7)',
 '15_min_bin_-_Misses_(7)',
 '15_min_bin_-_Mistakes_(7)',
 '15_min_bin_-_Correct_Rejections_(7)',
 '15_min_bin_-_Correction_Trial_Correct_Rejections_(7)',
 '15_min_bin_-_Correction_Trial_Mistakes_(7)']
df_tot=df_tot.drop(columns=drop_col)

## Caculate d' parameter
#### use the p concept to fill out zero value issue in time bin

In [23]:
#generate new column for hr, fr, d, si, c, ri, rp
from string import Template
#loop 6 different time bin
df_list=[df_tot]

df=df_tot
bin_number=["1", "2", "3", "4", "5", "6"]
for i in bin_number:

    #local 3 different time bin column
    hit='15_min_bin_-_Hits_(%s)'%(i)
    hit_c=df.loc[:, hit]

    miss = '15_min_bin_-_Misses_(%s)'%(i)
    miss_c=df.loc[:,miss]

    mistake = '15_min_bin_-_Mistakes_(%s)'%(i)
    mistake_c=df.loc[:,mistake]

    correct_rejection='15_min_bin_-_Correct_Rejections_(%s)'%(i)
    correct_rejection_c=df.loc[:,correct_rejection]

    center_iti_touches = '15_min_bin_-_Centre_ITI_Touches_(%s)'%(i)
    center_iti_touches_c=df.loc[:,center_iti_touches]

    Correction_Trial_Correct_Rejection = '15_min_bin_-_Correction_Trial_Correct_Rejections_(%s)'%(i)
    Correction_Trial_Correct_Rejection_c=df.loc[:,Correction_Trial_Correct_Rejection]

    Correction_Trial_Mistakes = '15_min_bin_-_Correction_Trial_Mistakes_(%s)'%(i)
    Correction_Trial_Mistakes_c = df.loc[:,Correction_Trial_Mistakes]

    #calculat hr and fr first
    df.loc[:,'hr_15min_bin_%s'%i]=hr_calcu(hit_c,miss_c)
    df.loc[:,'fr_15min_bin_%s'%i]=fr_calcu(mistake_c, correct_rejection_c)
df.to_csv("...\\tot_withzero_hr_fr.csv")  #replace the "..." to a full directory you want to save the file

df_zero = df[(df["hr_15min_bin_1"] == 0) |
                   (df["hr_15min_bin_2"] == 0) |
                   (df["hr_15min_bin_3"] == 0) |
                   (df["hr_15min_bin_4"] == 0) |
                   (df["hr_15min_bin_5"] == 0) |
                   (df["hr_15min_bin_6"] == 0) |
                   (df['fr_15min_bin_1'] == 0) |
                   (df['fr_15min_bin_2'] == 0) |
                   (df['fr_15min_bin_3'] == 0) |
                   (df['fr_15min_bin_4'] == 0) |
                   (df['fr_15min_bin_5'] == 0) |
                   (df['fr_15min_bin_6'] == 0)]

df_zero.to_csv("...\\tot_zero_hr_fr.csv", index=False) #replace the "..." to a full directory you want to save the file

for i in bin_number:
     #local 3 different time bin column
    hit='15_min_bin_-_Hits_(%s)'%(i)
    hit_c=df.loc[:, hit]

    miss = '15_min_bin_-_Misses_(%s)'%(i)
    miss_c=df.loc[:,miss]

    mistake = '15_min_bin_-_Mistakes_(%s)'%(i)
    mistake_c=df.loc[:,mistake]

    correct_rejection='15_min_bin_-_Correct_Rejections_(%s)'%(i)
    correct_rejection_c=df.loc[:,correct_rejection]

    center_iti_touches = '15_min_bin_-_Centre_ITI_Touches_(%s)'%(i)
    center_iti_touches_c=df.loc[:,center_iti_touches]

    Correction_Trial_Correct_Rejection = '15_min_bin_-_Correction_Trial_Correct_Rejections_(%s)'%(i)
    Correction_Trial_Correct_Rejection_c=df.loc[:,Correction_Trial_Correct_Rejection]

    Correction_Trial_Mistakes = '15_min_bin_-_Correction_Trial_Mistakes_(%s)'%(i)
    Correction_Trial_Mistakes_c = df.loc[:,Correction_Trial_Mistakes]
        
    #check if hr or fr is value 0, than replace the zero with p concept.
    for e in range(len(df)): 
        if df.iloc[e]['hr_15min_bin_%s'%i]==0:
            #display(df.loc[[e]])
            df.at[e, 'hr_15min_bin_%s'%i]=1/(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection])
        if df.iloc[e]['hr_15min_bin_%s'%i]==1:
            #display(df.loc[[e]])
            df.at[e, 'hr_15min_bin_%s'%i]=(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection]-1)/(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection])        
        if df.iloc[e]['fr_15min_bin_%s'%i]==0:
            #display(df.loc[[e]])
            df.at[e, 'fr_15min_bin_%s'%i]=1/(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection])
        if df.iloc[e]['fr_15min_bin_%s'%i]==1:
            #display(df.loc[[e]])
            df.at[e, 'fr_15min_bin_%s'%i]=(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection]-1)/(df.iloc[e][hit]+df.iloc[e][miss]+df.iloc[e][mistake]+df.iloc[e][correct_rejection]) 
        
    df.loc[:,'d_15min_bin_%s'%i]=d_calcu(df.loc[:,'hr_15min_bin_%s'%i], df.loc[:,'fr_15min_bin_%s'%i])
    df.loc[:,'si_15min_bin_%s'%i]=si_calcu(df.loc[:,'hr_15min_bin_%s'%i], df.loc[:,'fr_15min_bin_%s'%i])
    df.loc[:,'c_15min_bin_%s'%i]=c_calcu(df.loc[:,'hr_15min_bin_%s'%i], df.loc[:,'fr_15min_bin_%s'%i])
    df.loc[:,'ri_15min_bin_%s'%i]=ri_calcu(df.loc[:,'hr_15min_bin_%s'%i], df.loc[:,'fr_15min_bin_%s'%i])
    df.loc[:,'rp_15min_bin_%s'%i]=rp_calcu(center_iti_touches_c, hit_c, miss_c, mistake_c, correct_rejection_c, 
                                               Correction_Trial_Correct_Rejection_c, Correction_Trial_Mistakes_c)

df.loc[:, 'Hit_Rate'] =hr_calcu(df.loc[:, 'End_Summary_-_Hits_(1)'],df.loc[:, 'End_Summary_-_Misses_(1)'])
df.loc[:, 'False_Alarm_Rate'] =fr_calcu(df.loc[:, 'End_Summary_-_Mistakes_(1)'], df.loc[:, 'End_Summary_-_Correct_Rejections_(1)'])
for e in range(len(df)): 
    
    if df.iloc[e]['Hit_Rate']==0:
        trial_n=df.iloc[e]['End_Summary_-_Hits_(1)']+df.iloc[e]['End_Summary_-_Misses_(1)']+df.iloc[e]['End_Summary_-_Mistakes_(1)']+ df.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df.at[e, 'Hit_Rate']=1/(trial_n)
    if df.iloc[e]['Hit_Rate']==1:
        trial_n=df.iloc[e]['End_Summary_-_Hits_(1)']+df.iloc[e]['End_Summary_-_Misses_(1)']+df.iloc[e]['End_Summary_-_Mistakes_(1)']+ df.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df.at[e, 'Hit_Rate']=(trial_n-1)/(trial_n)
    
    if df.iloc[e]['False_Alarm_Rate']==0:
        trial_n=df.iloc[e]['End_Summary_-_Hits_(1)']+df.iloc[e]['End_Summary_-_Misses_(1)']+df.iloc[e]['End_Summary_-_Mistakes_(1)']+ df.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        df.at[e, 'False_Alarm_Rate']=1/(trial_n)
    if df.iloc[e]['False_Alarm_Rate']==1:
        trial_n=df.iloc[e]['End_Summary_-_Hits_(1)']+df.iloc[e]['End_Summary_-_Misses_(1)']+df.iloc[e]['End_Summary_-_Mistakes_(1)']+ df.iloc[e]['End_Summary_-_Correct_Rejections_(1)']
        print(str(e))
        df.at[e, 'False_Alarm_Rate']=(trial_n-1)/(trial_n)
        
df.loc[:, 'c'] =c_calcu(df.loc[:,'Hit_Rate'], df.loc[:,'False_Alarm_Rate'])
df.loc[:, 'd'] =d_calcu(df.loc[:,'Hit_Rate'], df.loc[:,'False_Alarm_Rate'])
df.loc[:, 'si'] =si_calcu(df.loc[:,'Hit_Rate'], df.loc[:,'False_Alarm_Rate'])
df.loc[:, 'ri'] =ri_calcu(df.loc[:,'Hit_Rate'], df.loc[:,'False_Alarm_Rate'])
df.loc[:, 'impuls'] =100*(df.loc[:,'End_Summary_-_Centre_ITI_Touches_(1)']/df.loc[:,'trial_by_trial_anal_-_ITI'])

In [24]:
#check if inf in d' exist
df_inf=df[['d_15min_bin_1', 'd_15min_bin_2','d_15min_bin_3','d_15min_bin_4','d_15min_bin_5','d_15min_bin_6']]
count = np.isinf(df_inf).values.sum()
print("It contains " + str(count) + " infinite values")

It contains 0 infinite values


In [25]:
df.to_csv("...\\df_tot.csv") #replace the "..." to a full directory you want to save the file

In [ ]:
df_tot=df

In [ ]:
#check zero row
def get_columns_with_zeros(dataframe):
    zero_columns = []
    
    for column in dataframe.columns:
        if (dataframe[column] == 0).any():
            zero_columns.append(column)
    
    return zero_columns

get_columns_with_zeros(df_tot)

# built feature list

In [ ]:
hit_list=[]
false_list=[]
miss_list=[]
hr_list=[]
fr_list=[]
d_list=[]
si_list=[]
c_list=[]
ri_list=[]
rp_list=[]
cor_latency_mean=[]
incor_latency_mean=[]
reward_latency_mean=[]
cor_latency_sd=[]
incor_latency_sd=[]
reward_latency_sd=[]
corr_reject_list=[]

In [ ]:
a_list=[hit_list,false_list,miss_list, corr_reject_list,
        hr_list,fr_list,d_list,si_list,c_list,ri_list,rp_list,
        cor_latency_mean,incor_latency_mean,reward_latency_mean,cor_latency_sd,incor_latency_sd,reward_latency_sd]

feature_list=["15_min_bin_-_Hits_(%s)","15_min_bin_-_Mistakes_(%s)","15_min_bin_-_Misses_(%s)",'15_min_bin_-_Correct_Rejections_(%s)',
"hr_15min_bin_%s","fr_15min_bin_%s","d_15min_bin_%s","si_15min_bin_%s","c_15min_bin_%s","ri_15min_bin_%s","rp_15min_bin_%s",
"corr_latency_bin_%s","incorr_latency_bin_%s","reward_latency_bin_%s","corr_latency_bin_%s_sd","incorr_latency_bin_%s_sd","reward_latency_bin_%s_sd"]

for a, i in zip(a_list, feature_list):
    for j in ["1", "2", "3", "4", "5","6"]:
        a.append(i%(j))

### prepare df for plot

In [ ]:
df_tot_mean=df_tot.groupby(by=['id','gander', 'cohort'], as_index=False).mean()

In [ ]:
df_list=[df_tot_mean]

for j in df_list:
    for i in reward_latency_mean:
        j[i] = (j[i].astype(str).str.split()).apply(lambda x: float(x[0].replace(',', '')))

In [ ]:
df_tot_melt=df_tot_mean.melt(id_vars=[
 'id','gander', 'cohort'
 ], var_name="param", value_name="value").copy()

In [ ]:
df_tot_melt[(df_tot_melt['param'].str.contains("d")) & (df_tot_melt['value']==0)]

#### check inf data in the dataframe

In [ ]:
df_list=[df_tot_melt]
name_list=["df_tot_melt"]
for j, i in zip(df_list, name_list):
    print()
    print(i)
    print("printing the count of infinity values")
    count = np.isinf(j["value"]).values.sum()
    print("It contains " + str(count) + " infinite values")
    
    print()
    print("printing row index with infinity ")
  
    r = j[np.isinf(j["value"])]
    display(r[r["param"].str.contains("15")])

    f = j[j["value"]==0]
    display(f[f["param"].str.contains("15")])

### plot 15min bin

In [ ]:
def normal(mean, std, histmax=False, color="black"):
    x = np.linspace(mean-4*std, mean+4*std, 200)
    p = stats.norm.pdf(x, mean, std)
    if histmax:
        p = p*histmax/max(p)
    z = plt.plot(x, p, color, linewidth=2)

In [ ]:
df_list=[df_tot_melt]
folder_list=["tot_15"]

for a, b  in zip(df_list, folder_list):
    
    for i in a_list:
            
        df=a[(a["param"].isin(i))].copy()
        plot=sns.pointplot(x='param', y='value', hue='gander', hue_order=['male', 'female'],palette=['royalblue', 'coral'], 
                           data=df, errorbar=('ci', 68)) 
        plot.set(xlabel="")
        plot.set(ylabel="")
        title=i[0]+'_'+b
        plt.title(title)
        print(title)
        plt.xticks(rotation = 45, ha='right')
        plt.legend(bbox_to_anchor=(1.4, 1),borderaxespad=0)
        
        save_path = "...\\fig\\%s"%b #replace the "..." to a full directory you want to save the file 
        if os.path.isdir(save_path) == False:
            os.mkdir(save_path) 
        plt.savefig(save_path+'\%s.png'%(title), dpi=800, bbox_inches="tight")
        plt.close()
        df.to_csv(save_path+'\%s.csv'%(title), index=False)    
